In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import chemiscope
from widget_code_input import WidgetCodeInput
from ipywidgets import Textarea
from iam_utils import *
import ase
from ase.io import read, write
import itertools
from copy import deepcopy

In [ ]:
#### AVOID folding of output cell 

In [ ]:
%%html

<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:4000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [ ]:
data_dump = WidgetDataDumper(prefix="ex_02")
display(data_dump)

_Reference textbook / figure credits: Charles Kittel, *Introduction to solid-state physics", Chapter 1_

# Unit cells, fractional coordinates, lattices

A periodic structure is defined by a *lattice* and an a-periodic *repeat unit*. The lattice is a periodic set of points generated by all integer combinations of three *unit cell vectors* $\mathbf{a}_{1,2,3}$, i.e. 

$$
\mathbf{T} = u_1 \mathbf{a}_1 +  u_2 \mathbf{a}_2 +  u_3 \mathbf{a}_3, \quad u_{1,2,3} \in \mathbb{Z}
$$

The repeat unit, or *basis* is defined by the coordinates $\{x_i, y_i, z_i\}$ of a (usually small) number of atoms that sit in arbitrary positions within the unit cell. These are often given in *fractional coordinates* $\{s_{i1}, s_{i2}, s_{i3}\}$, such that the set of all atoms in the crystal is generated by 

$$
(u_1+s_{i1}) \mathbf{a}_1 +  (u_2+s_{i2}) \mathbf{a}_2 + (u_3+s_{i3})
$$

with the $u_{1,2,3}$ ranging over all positive and negative integers, and $i$ ranging over the number of atoms in the basis.

There are 14 types of lattices known as _Bravais lattices_ in three dimensions, that are distinguished by the symmetry group of the lattice points. If you need to refresh your fundamentals of crystallography, and don't remember what a _face centered cubic_ or _body centered cubic_ lattice is, the [wikipedia page](https://en.wikipedia.org/wiki/Bravais_lattice) is excellent. Note also that the crystal structure (lattice+basis) can have more complicated symmetries, forming the 230 [space groups](https://en.wikipedia.org/wiki/Space_group)

Now, consider the structure below. This is just a finite set of atoms, and coordinates are listed individually. You can click on the atoms in the viewer and see its coordinates by clicking on the info panel below the viewer.

In [ ]:
positions = np.array( [[x,y,z] for x,y,z in itertools.product([0, 5, 10, 15],[0, 5, 10, 15],[0, 5, 10, 15]) ])
ase_cube = ase.Atoms("Ga64", positions=positions)

properties = {}
properties["index"] = {
                "target": "atom",
                "values": list(range(1, len(ase_cube)+1)),
            }

properties["coordinates"] = {
                "target": "atom",
                "values": positions,
            }

cs = chemiscope.show([ase_cube], properties=properties, mode="structure",                      
                     environments=chemiscope.all_atomic_environments([ase_cube], cutoff=40),
                     settings={"structure":[{"spaceFilling": False, "environments": {"cutoff": 40}}]}
                    )
display(cs)


<span style="color:blue"> **01** Write a function that returns the lattice vectors and a basis that generates the periodic structure that continues to infinity the motif above. Try also to shift rigidly the basis atoms. Does the resulting crystal change in a significant way? </span>

_NB: the validation code only tests for the primitive cell - you can come up with correct structures that will be marked as incorrect. Trust your own judgment (and look carefully at the visualization)_

In [ ]:
ex01_wci = WidgetCodeInput(
        function_name="sc_lattice_basis", 
        function_parameters="",
        docstring="""
Returns the lattice vectors and the basis (in fractional coordinates) that generates a simple cubic structure.

:return: Three lattice vectors and a list of basis coordinates
""",
            function_body="""
# Write your solution, then click on the button below to update the plotter 
# and check against the reference value

a1 = []
a2 = []
a3 = []
basis = [[]]

return a1, a2, a3, basis
"""
        )

data_dump.register_field("ex01-function", ex01_wci, "function_body")
def ex01_updater():
    a1, a2, a3, basis = ex01_wci.get_function_object()()
    positions = np.asarray(basis)
    h = np.asarray([a1,a2,a3])
    
    structure = ase.Atoms('Ga'+str(len(basis)), positions=positions@h.T, cell=[a1, a2, a3])
    display(chemiscope.show(frames = [structure], mode="structure", 
                            settings={"structure":[{"unitCell":True,"supercell":{"0":3,"1":3,"2":3}}]}
                           ))

def match_lattice(a, b):
    a1, a2, a3, basis = b;
    
    return np.allclose([a1,a2,a3], np.eye(3)*5) and np.asarray(basis).shape==(1,3)
    
"""ref_values = {
        (): ase.Atoms("CNH6", positions=[[ 1.,   -0.,   -0.01],
 [ 2.52, -0.01,  0.  ],
 [ 0.6 ,  1.02, -0.  ],
 [ 0.59, -0.52,  0.88],
 [ 0.6 , -0.51, -0.9 ],
 [ 2.92,  0.5 ,  0.89],
 [ 2.93,  0.51, -0.88],
 [ 2.92, -1.03, -0.  ]]) 
       }, ref_match = match_structure, """    
ex01_wcc = WidgetCodeCheck(ex01_wci, ref_values = { (): ([5,0,0],[0,5,0],[0,0,5],[[0,0,0]]) },
                           ref_match = match_lattice,
                           demo=WidgetUpdater(updater=ex01_updater))    
display(ex01_wcc)

In [ ]:
ex01_txt = Textarea("enter any additional comment", layout=Layout(width="100%"))
data_dump.register_field("ex01-answer", ex01_txt, "value")
display(ex01_txt)

# Primitive cell, supercells, conventional cells

It is always possible to give different descriptions to the same crystal structure. For instance, for a given cell and basis one can always specify a _supercell_ i.e. a multiple of repeat units of the original cell, with a correspondingly larger basis. This basis contains "hidden" symmetries, meaning that it would be possible to describe the same structure with a smaller unit cell. The smallest possible cell is called a _primitive_ (or minimal) cell. 

For instance, the figure below shows three different choices of unit cell for a face-centered cubic lattice. All cells describe the same structure! The first structure is the primitive cell, and the third the conventional cells that reveals more clearly the origin of the name of this lattice.

<img src="figures/fcc-cells.png" width="800"/>

The following visualizer shows a collection of 9 crystal structures, containing either one or two chemical species. Look at them, and try to understand what type of lattice they belong to. You can visualize the environment of each atom within an adjusable cutoff, which may help you appreciate the 3D nature of the structure.

In [ ]:
ase_crystals = read('data/crystals.xyz',":")

properties = {}
properties["lattice vectors a1"] = {
                "target": "structure",
                "values": np.asarray([str(list(a.cell[0])) for a in ase_crystals]),
            }
properties["lattice vectors a2"] = {
                "target": "structure",
                "values": np.asarray([str(list(a.cell[1])) for a in ase_crystals]),
            }
properties["lattice vectors a3"] = {
                "target": "structure",
                "values": np.asarray([str(list(a.cell[2])) for a in ase_crystals]),
            }

properties["basis"] = {
                "target": "atom",
                "values": np.vstack([a.positions for a in ase_crystals]),
            }


cs = chemiscope.show(ase_crystals, properties=properties, mode="structure",                      
                     environments=chemiscope.all_atomic_environments(ase_crystals),
                     settings={"structure":[{"bonds":True, "unitCell":True,"supercell":{"0":3,"1":3,"2":3},
                                            "environments": {"cutoff": 6}}]}                    
                    )

def update_co(change):
    cs.settings={"structure": [{"environments": {"cutoff": pb.value['co']}}]}
pb = WidgetParbox(onchange=update_co, co=(6.,1,12,0.1, r"environment cutoff / Å"))
display(VBox([pb,cs]))


<span style="color:blue"> **02** Each of the structures above are either face-centered (fcc), body-centered (bcc) or simple cubic (sc). Write down in the box below what is the Bravais lattice for each structure, and the size of the basis used to describe the structure in each frame. </span>

_NB: pay attention: (1) you are asked about the symmetry of the **lattice**: if you just look at the cell, you may be misled; (2) the number of atoms included in the basis depend on the choice of cell._

In [ ]:
ex02_txt = Textarea("structure 1:  lattice: XXX, basis size: YYY\n ....", layout=Layout(width="100%"))
data_dump.register_field("ex02-answer", ex02_txt, "value")
display(ex02_txt)

Using a supercell is not only a way of obtaining a more convenient/intuitive view of a crystal structure. It can also be useful to represent real materials, in which the ideal periodicity of the crystal is broken, e.g. because there are defects, interfaces, or simply disorder. In these cases, one often starts from a small unit cells and explicitly repeats it many times. _This is different from just replicating the unit cell for visualization purposes: the cell is also enlarged, and the coordinates of all atoms inside the unit cell can be adjusted independently_

ASE provides convenient utilities to replicate a structure. If `structure` is an `ase.Atoms` object, then
`structure.repeat((nx, ny, nz))` will replicate the structure the given number of times along each of the axes. 

<span style="color:blue"> **03** Write code to generate a supercell for _fcc_ aluminum, using the number of repeat units as an argument, and a lattice parameter of 4Å. You can use the `repeat` ASE command, but if you feel adventurous you may as well write a replicate function yourself. </span>

_NB: the self-test function can generate false errors because it expects a specific order of the basis atoms. Don't worry too much if it tells you one test has not passed!_

In [ ]:
ex03_wci = WidgetCodeInput(
        function_name="al_supercell", 
        function_parameters="nrep",
        docstring="""
Returns an ASE object describing a supercell built by replicating `nrep` times along each direction a conventional (4-atoms)
fcc unit cell for aluminum. Use a lattice parameter of 4 Å.

:return: The replicated-cell ASE object
""",
            function_body="""
# Write your solution, then click on the button below to update the chemiscope viewer
import ase

al4 = ase.Atoms("Al4", pbc=True)
al4.cell = [ ... ] # lattice parameters of the conventional unit cell
al4.positions = [[, ,], [, ,], [, ,], [, ,]] # positions of the 4 atoms in the conventional fcc cell


# empty atom
al_replicated =  ... 

return al_replicated
"""
        )

data_dump.register_field("ex03-function", ex03_wci, "function_body")
def ex03_updater():
    al_multi = ex03_wci.get_function_object()(ex03_wp.value['nrep'])
    display(VBox([ex03_wp,chemiscope.show(frames = [al_multi], mode="structure", 
                            settings={"structure":[{"unitCell":True}]}
                           )]) )

def match_lattice(a, b):    
    return np.allclose(b.cell, a[0]) and (True if a[1] is None else np.allclose(b.positions-b.positions[0], a[1])) 
    
ex03_wp = WidgetParbox(nrep=(2, 1, 4, 1, r"$n_{\mathrm{repeat}}$ (must click on update button!)"))    
ex03_wcc = WidgetCodeCheck(ex03_wci, ref_values = { 
    (1,): ([[4,0,0],[0,4,0],[0,0,4]], [[0,0,0],[2,2,0],[2,0,2],[0,2,2]]),
    (2,): ([[8,0,0],[0,8,0],[0,0,8]], None) 
},
                           ref_match = match_lattice,
                           demo=WidgetUpdater(updater=ex03_updater))
display(ex03_wcc)

# Lattice planes and surfaces 

The general notation used to define lattice planes is somewhat contrived (because it actually relates to the reciprocal lattice!). A plane is defined by three points, so one can define a lattice plane by picking three points that are multiples of the unit cell vectors, $(n_1 \mathbf{a_1}, n_2 \mathbf{a_2}, n_3 \mathbf{a_3})$. The plane is determined by the reciprocals of the $n_{1,2,3}$, multiplied by their least common multiple so as to obtain three integer indices $(h k l)$. For instance, a plane that intercepts the lattice axes with $n_{1,2,3} = (4,3,1)$ has Miller indices $(3,4,12)$. _Only in a cubic lattice_ the plane identified by (h k l) has $h \mathbf{a_1} + k \mathbf{a_2} + l \mathbf{a_3}$ as a plane normal. 

If you need a more detailed discussion, see Chapter 1 of _Kittel_, or the [Wikipedia page on Miller indices](https://en.wikipedia.org/wiki/Miller_index) that has some good figures. 

In the widget below, you can see a large supercell of _fcc_ aluminum. You can select different Miller indices, and see the atoms that belong to one of the lattice planes highlighted in a different color. Experiment a bit and note how e.g. `(2,2,4)` is equivalent to `(1,1,2)`, etc. Note also how the density of atoms on low-index planes is higher than on high-index planes.

In [ ]:
fcc_al = ase.lattice.cubic.FaceCenteredCubic('Al')
al_supercell = fcc_al.repeat((4,4,4))

normal_plane = np.array([1,0,0])
origin = np.diagonal(al_supercell.cell)/2
distances = np.abs((al_supercell.positions-origin) @ normal_plane) / np.linalg.norm(normal_plane)
atoms_on_plane = np.where( distances < 1e-5 )[0]
al_supercell.numbers[atoms_on_plane] = 12


cs = chemiscope.show([al_supercell], properties=properties, mode="structure",                      
                     settings={"structure":[{"spaceFilling": False, "unitCell":True}]}
                    )

def update_surface_cut(change):
    al_supercell.numbers[:] = 13
    normal_plane = np.array([pb.value['h'],pb.value['k'],pb.value['l']])    
    origin = np.diagonal(al_supercell.cell)/2
    distances = np.abs((al_supercell.positions-origin) @ normal_plane) / np.linalg.norm(normal_plane)
    atoms_on_plane = np.where( distances < 1e-6 )[0]
    al_supercell.numbers[atoms_on_plane] = 12
    global cs 
    settings = cs.settings
    cs.close()
    cs = chemiscope.show([al_supercell], properties=properties, mode="structure",                      
                         settings=cs.settings
                        )
    display(cs)


    #cs.settings={"structure": [{"environments": {"cutoff": pb.value['co']}}]}
pb = WidgetParbox(onchange=update_surface_cut, h=(1,0,3,1,r"h"), k=(0,0,3,1,r"k"), l=(0,0,3,1,r"l"))
display(VBox([pb,cs]))


<span style='color:blue'> **04** Activate the visualization of multiple replicas in the visualizer above. Are the planes continuous across the edges of the supercell, if you pick a `(1,0,0)` plane? And what happens if you pick `(1,1,0)`? </span>

In [ ]:
ex04_txt = Textarea("enter your answer", layout=Layout(width="100%"))
data_dump.register_field("ex04-answer", ex04_txt, "value")
display(ex04_txt)

Now let's create a supercell with an actual surface. While there are tools to do this for more complicated scenarios (see e.g. [here](https://wiki.fysik.dtu.dk/ase/ase/build/surface.html#create-specific-non-common-surfaces) for some examples using ASE), we will do this manually to understand better what is going on. 

If the surface is aligned with one of the lattice parameters, it is sufficient to first replicate the unit cell a number of times, and then artificially enlarge one of the edges of the unit cell. This will leave some empty space between the top and bottom layers of the structure, effectively leaving some atoms in contact with vacuum. 

<img src="figures/slab-100.png" width="400"/>

Note you always create _two_ surfaces, because you are still dealing with a periodic structure. This is usually referred to as a _slab geometry_. Note also that for complicated crystals with a basis, the unit cell can be cut at different positions, so that there can be multiple different surfaces corresponding to the same lattice direction, depending on the position of the cut. 

<span style='color:blue'> **05** Write a function that creates a supercell for _fcc_ aluminum with 4x4x2 replicas along the three directions. Use a conventional unit cell and a lattice parameter of 4Å. Modify the `cell` of the resulting ASE structure to add 20Å of vacuum along the $\mathbf{a}_3$ direction, creating a slab geometry with a surface along `(0,0,1)` </span>

_NB: you can copy most of the code from exercise 03._

In [ ]:
ex05_wci = WidgetCodeInput(
        function_name="al_slab", 
        function_parameters="nrep, nslab, vacuum",
        docstring="""
Returns an ASE object describing Al (100) surfaces using a slab geometry with nrep x nrep x nslab cells,
and a vacuum region along z. Use a lattice parameter of 4 Å.

:return: The ASE object describing the slab geometry
""",
            function_body="""
# Write your solution, then click on the button below to update the chemiscope viewer
import ase

al4 = ase.Atoms("Al4", pbc=True)
al4.cell = [ ... ] # lattice parameters of the conventional unit cell
al4.positions = [[, ,], [, ,], [, ,], [, ,]] # positions of the 4 atoms in the conventional fcc cell


# empty atom
al_replicated =  ... 
# create gap

return al_replicated
"""
        )

data_dump.register_field("ex05-function", ex05_wci, "function_body")
def ex05_updater():
    al_multi = ex05_wci.get_function_object()(ex05_wp.value['nrep'], ex05_wp.value['nslab'], ex05_wp.value['vacuum'])
    display(VBox([ex05_wp,chemiscope.show(frames = [al_multi], mode="structure", 
                            settings={"structure":[{"unitCell":True}]}
                           )]) )

def match_lattice(a, b):    
    return np.allclose(b.cell, a[0]) and (True if a[1] is None else np.allclose(b.positions-b.positions[0], a[1])) 
    
ex05_wp = WidgetParbox(
    nrep=(4, 1, 6, 1, r"$n_{\mathrm{rep}}$ (click on button to update!)"),
    nslab=(2, 1, 6, 1, r"$n_{\mathrm{slab}}$"),
    vacuum=(20., 0, 40, 0.5, r"$L_{\mathrm{vacuum}}$")
    )
ex05_wcc = WidgetCodeCheck(ex05_wci, ref_values = { 
    (1, 1, 10): ([[4,0,0],[0,4,0],[0,0,14]], [[0,0,0],[2,2,0],[2,0,2],[0,2,2]]),
    (4, 2, 11): ([[16,0,0],[0,16,0],[0,0,19]], None) 
},
                           ref_match = match_lattice,
                           demo=WidgetUpdater(updater=ex05_updater))
display(ex05_wcc)

Creating a `(1,1,1)` surface is somewhat more complicated, because the conventional unit cell is not aligned properly. One way to create a slab with the appropriate orientation is to create a _primitive_ face-centered cell and to create the slab by _extending_ one of the cell vectors. 

<span style='color:blue'> **06** Write a function that creates a supercell for _fcc_ aluminum, starting with the *primitive* cell replicated as 8x8x2 along the lattice vectors. Then _multiply_ by four $\mathbf{a}_3$. Visualize the resulting structure</span>

In [ ]:
ex06_wci = WidgetCodeInput(
        function_name="al_slab_111", 
        function_parameters="nrep, nslab, zscale",
        docstring="""
Returns an ASE object describing Al (111) surfaces using a slab geometry with 
nrep x nrep x nslab copies of the primitive cell. Vacuum must be created by 
elongating one of the lattice vectors by a factor zscale. Use a lattice parameter of 4 Å.

:return: The ASE object describing the slab geometry
""",
            function_body="""
# Write your solution, then click on the button below to update the chemiscope viewer
import ase

al = ase.Atoms("Al", pbc=True)
al.cell = [ ... ] # lattice parameters of the *primitive* unit cell
al.positions = [[0,0,0]] # just one-atom basis!


# empty atom
al_replicated =  ... 
# create gap

return al_replicated
"""
        )

data_dump.register_field("ex06-function", ex06_wci, "function_body")
def ex06_updater():
    al_multi = ex06_wci.get_function_object()(ex06_wp.value['nrep'], ex06_wp.value['nslab'], ex06_wp.value['zscale'])
    display(VBox([ex06_wp,chemiscope.show(frames = [al_multi], mode="structure", 
                            settings={"structure":[{"unitCell":True}]}
                           )]) )

def match_lattice(a, b):    
    return np.allclose(b.cell, a[0]) and (True if a[1] is None else np.allclose(b.positions-b.positions[0], a[1])) 
    
ex06_wp = WidgetParbox(
    nrep=(4, 1, 6, 1, r"$n_{\mathrm{rep}}$ (click on button to update!)"),
    nslab=(2, 1, 6, 1, r"$n_{\mathrm{slab}}$"),
    zscale=(4., 1., 10, 0.5, r"$z_{\mathrm{scale}}$")
    )
ex06_wcc = WidgetCodeCheck(ex06_wci, ref_values = { 
    (1, 1, 1): ([[2,2,0],[2,0,2],[0,2,2]], [[0,0,0]]),
    (1, 2, 2): ([[2,2,0],[2,0,2],[0,8,8]], None) 
},
                           ref_match = match_lattice,
                           demo=WidgetUpdater(updater=ex06_updater))
display(ex06_wcc)

<span style="color:blue"> **07** What is the direction of the primitive lattice parameters relative to the conventional _fcc_ index system? Is the surface normal parallel to the $\mathbf{a}_3$ cell vector? 
In order to create an easier to visualize structure, one often builds a unit cell that is orthorhombic and has one axis that is parallel to the surface normal. Can you come up with three low-Miller-indices directions that are mutually orthogonal and could be used to define an appropriate unit cell?
</span>

_NB: The last point is not entirely trivial, think about it but don't get too stressed if you don't manage to create the appropriate cell. If you are *really* into these geometric problems, you can also try to derive an actual unit cell, including also the basis._

In [ ]:
ex07_txt = Textarea("write your answer", layout=Layout(width="100%"))
data_dump.register_field("ex07-answer", ex07_txt, "value")
display(ex07_txt)

# Reciprocal lattice in 2D

From here on, we work exclusively with 2D lattices,

$$
\mathbf{T} = u_1 \mathbf{a}_1 + u_2 \mathbf{a}_2, \quad \mathbf{a}_{1,2} \in \mathbb{R}^2, u_{1,2} \in Z
$$

because they allow for simpler visualization of the core concepts. Most of the concepts we develop and demonstrate, however, apply equally well to actual 3D systems. 

The idea of the reciprocal lattice is deeply linked to the idea of scatterig, and constructive interference. A plane waves $e^{\mathrm{i} \mathbf{k}\cdot \mathbf{x}}$ originating at the origin will be in phase with similar waves originating at all other lattice points if its wavevector $\mathbf{k}$ satisfy the phase relation $\mathbf{k}\cdot \mathbf{T} = 2\pi n$, where $n$ is an integer, for each and every lattice vector. 

We can build a _reciprocal lattice_ that consist of integer combinations of basis vectors $\mathbf{b}_{1,2}$

$$
\mathbf{G} = v_1 \mathbf{b}_1 + v_2 \mathbf{b}_2, \quad \mathbf{b}_{1,2} \in \mathbb{R}^2, v_{1,2} \in Z
$$

such that every reciprocal lattice point is a wavevector that results in a complete in-phase scattering from the direct lattice. To do this, we need relationships to hold between the lattice vectors: $\mathbf{b}_{1,2} \cdot \mathbf{a}_{1,2} = 2\pi$, $\mathbf{b}_{1,2}\cdot \mathbf{a}_{2,1} = 0$. Thus, 
$\mathbf{b}_1$ must be orthogonal to $\mathbf{a}_2$ and $\mathbf{b}_2$ must be orthogonal to $\mathbf{a}_1$.

We can achieve this (including also normalization that ensure the correct scaling $\mathbf{b}_{1,2} \cdot \mathbf{a}_{1,2} = 2\pi$, with the definition

$$ \mathbf{b}_1 = 2\pi\frac{\mathbf{R}\mathbf{a}_2}{\mathbf{a}_1\cdot\mathbf{R}\mathbf{a}_2}, \quad \mathbf{b}_2 = 2\pi\frac{\mathbf{R}\mathbf{a}_1}{\mathbf{a}_2\cdot\mathbf{R}\mathbf{a}_1},\quad\quad\mathbf{R} = \begin{bmatrix}0 & -1\\ 1 & 0\end{bmatrix}$$

where $\mathbf{R}$ is a $\pi/2$ rotation. 

<span style="color:blue"> **08** What are the reciprocal lattice vectors for a 2D rectangular Bravais lattice with lattice vectors $(1,0)$ and $(0,2)$?
</span>

In [ ]:
ex08_txt = Textarea("The reciprocal primitive vectors are:\n b_1 = ... , b_2 = ...", layout=Layout(width="100%"))
data_dump.register_field("ex08-answer", ex08_txt, "value")
display(ex08_txt)

<span style="color:blue"> **09** Write a function that computes the reciprocal lattice vectors given $\mathbf{a}_{1,2}$. Use the demo widget to experiment and get an intuitive understanding of what's going on. 
</span>

In [ ]:
# TODO AG: make an exercise where they build b1, b2 starting from
# a1, a2, and have a demo box that shows two panels, one with the direct lattice
# and one with the reciprocal lattice
# QUESTION@michele I put them for this exercise real and reciprocal in the same plot, 
#                  let me know if you still prefer two separate as in the next one.
#                  I think for this task it could be useful to see them overlapped

ex09_wci = WidgetCodeInput(
        function_name="reciprocal_lattice", 
        function_parameters="a1, a2",
        docstring="""
Return the 2D reciprocal unit cell vectors.

:param a1: unit cell vector a1 
:param a2: unit cell vector a2

:return: reciprocal lattice unit cell vectors
""",
        function_body="""

import numpy as np
from numpy import pi

# matrix-matrix and matrix-vector multiplication can be doen with @
# For example multiplying vector a with matrix R can be done with
# R @ a
R = np.array([[0,-1],[1,0]])  # <- this converts a nested list to a numpy array

b1 = 2*pi*a1 # change to correct solution
b2 = 2*pi*a2 # change to correct solution

### END SOLUTION
return b1, b2
"""
        )

data_dump.register_field("ex09-function", ex09_wci, "function_body")

In [ ]:
def plot_reciprocal_and_real_lattice(axes, a11, a12, a21, a22):
    a1 = np.array([a11, a12])
    a2 = np.array([a21, a22])
    
    b1, b2 = ex09_wci.get_function_object()(a1, a2)

    A = np.array([a1, a2])
    B = np.array([b1, b2])
    
    # ! has to be mod 2 integer !
    lattice_size = 120    
    real_lattice = (np.mgrid[:lattice_size,:lattice_size].T @ A).reshape(-1, 2)
    real_lattice -= (np.array([lattice_size//2,lattice_size//2]) @ A).reshape(-1, 2)
    
    reciprocal_lattice = (np.mgrid[:lattice_size,:lattice_size].T @ B).reshape(-1, 2)
    reciprocal_lattice -= (np.array([lattice_size//2,lattice_size//2]) @ B).reshape(-1, 2)
    
    s = 20
    axes[0].scatter(real_lattice[:,0], real_lattice[:,1], color='red', s=s)
    axes[1].scatter(reciprocal_lattice[:,0]/(2*np.pi), reciprocal_lattice[:,1]/(2*np.pi), color='darkblue', s=s)

    head_length = 0.5
    head_width = 0.2
    width = 0.05
    lw = 1
    alpha_unit_cell_vectors = 0.8

    axes[1].arrow(0, 0, b1[0]/(2*np.pi), b1[1]/(2*np.pi), lw=lw,
             head_width=head_width, head_length=head_length,
             width=width,
             alpha=alpha_unit_cell_vectors,
             length_includes_head=True,
             fc='darkblue', ec='black')

    axes[1].arrow(0, 0, b2[0]/(2*np.pi), b2[1]/(2*np.pi), lw=lw,
             head_width=head_width, head_length=head_length,
             width=width,
             alpha=alpha_unit_cell_vectors,
             length_includes_head=True,
             fc='darkblue', ec='black')

    
    axes[0].arrow(0, 0, a1[0], a1[1], lw=lw,
             head_width=head_width, head_length=head_length,
             width=width-0.1,
             alpha=alpha_unit_cell_vectors,
             length_includes_head=True,
             fc='red', ec='palevioletred')

    axes[0].arrow(0, 0, a2[0], a2[1], lw=lw,
             head_width=head_width, head_length=head_length,
             width=width-0.05,
             alpha=alpha_unit_cell_vectors,
             length_includes_head=True,
             fc='red', ec='palevioletred')

    axes[0].set_title('real space')
    axes[0].set_xlim(-5,5)
    axes[0].set_ylim(-5,5)
    axes[0].set_xlabel("$x$ / Å")
    axes[0].set_ylabel("$y$ / Å")
    
    axes[1].set_title('reciprocal space')
    axes[1].set_xlim(-5,5)
    axes[1].set_ylim(-5,5)
    axes[1].set_xlabel("$k_x/2\pi$ / Å$^{-1}$")
    axes[1].set_ylabel("$k_y/2\pi$ / Å$^{-1}$")


In [ ]:
fig_ax = plt.subplots(1, 2, figsize=(7.5,3.8), tight_layout=True)
ex09_wp = WidgetPlot(plot_reciprocal_and_real_lattice, 
               WidgetParbox(a11 = (1., -4, 4, 0.1, r'$a_{11} / Å$'),
                            a12 = (0., -4, 4, 0.1, r'$a_{12} / Å$'),
                            a21 = (0., -4, 4, 0.1, r'$a_{21} / Å$'),
                            a22 = (2., -4, 4, 0.1, r'$a_{22} / Å$')
                           ),
                     fig_ax=fig_ax)
ex09_wcc = WidgetCodeCheck(ex09_wci, ref_values={}, demo=ex09_wp)
display(ex09_wcc)

# Diffraction from a lattice

To understand diffraction, you need to consider that scattering is a process involving an incoming wave, $e^{\mathrm{i} \mathbf{k}_{\mathrm{in}} \cdot \mathbf{x}}$. Each lattice point will be hit by the wave with a different phase 
$e^{\mathrm{i} \mathbf{k}_{\mathrm{in}} \cdot \mathbf{T}}$ and act as a spherical scatterer. The detector will be in a given position $\mathbf{R}_d$, and if it is sufficiently far it pick up a plane wave in that direction, with wavevector $\mathbf{k}_{\mathrm{out}}$. Depending on the position of the scattering point, it will accumulate a further phase $e^{-\mathrm{i} \mathbf{k}_{\mathrm{out}} \cdot \mathbf{T}}$ (the minus coming from the fact the distance traveled will be $~(\mathbf{R}_d - \mathbf{T})$. The scheme below, drawn for arbitrary position of the scattering centers, applies clearly also to the case when scattering occurs from lattice points

<img src="figures/scattering.png" width="600"/>

The condition to have constructive interference is therefore to have $\mathbf{k}=\mathbf{k}_\mathrm{in}-\mathbf{k}_\mathrm{out}$ be a vector of the reciprocal lattice.

For typical diffraction experiments, the scattering process is elastic, meaning that the modulus of the wavevector does not change during the interaction. Furthermore, the modulus of the wavevector depends on the source of radiation used in the experiment. 

<span style="color:blue"> **10** what are the smallest and largest reciprocal lattice vectors that can be observed with electromagnetic radiation with wavelength $\lambda$? Think of the geometry of the experiment, and take the modulus of the wavevector to be $2\pi/\lambda$. 
</span>

In [ ]:
ex10_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex10-answer", ex10_txt, "value")
display(ex10_txt)

The widget below allows you to experiment with the geometry of a scattering experiment. You can define a direct lattice, the wavelength of the incoming light, and the angle between incoming and outgoing beam. 
The scheme that is drawn around the origin is the construction for the Ewald circle (2D equivalent of the Ewald sphere). There may be scattering only if the scattering vector $\mathbf{k}$ coincides with a point of the reciprocal lattice. 

In [ ]:
# TODO AG make a widget similar to the one to visualize the reciprocal 
# lattice. Options are the real-space lattice vectors as above. You now 
# add also a wavelength, and the angle between the incoming and outcoming wavevector.
# in the reciprocal lattice panel you show the incoming wavevector along the X axis, 
# and the outcoming one in whatever direction it has to be. You draw the Laue 
# sphere corresponding to k. 

In [ ]:
def plot_braggs_reflection(axes, a11, a12, a21, a22, phi, wavelength, two_theta):
    def rot2d(angle):
        return np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
    rot_phi = rot2d(phi)
    a1 = np.array([a11, a12]) @ rot_phi
    a2 = np.array([a21, a22]) @ rot_phi
    
    # reuse 
    plot_reciprocal_and_real_lattice(axes, a1[0], a1[1], a2[0], a2[1])
    
    # tried to plot phi vector, but results in too many vectors
    #phi_vector = rot_phi @ np.array([0, 1])*2
    #axes[0].arrow(7.8, 7.7, phi_vector[0], phi_vector[1], lw=lw,
    #         head_width=0.5, head_length=head_length,
    #         width=0.05,
    #         length_includes_head=True,
    #         fc='black', ec='black')
    
    head_length = 0.5
    head_width = 0.2
    width = 0.05
    lw = 1
    alpha_unit_cell_vectors = 0.8
    
    k_in = 2*np.pi*np.array([-1,0])/wavelength /(2*np.pi) #<- scale by 2pi so it is consistent with reciprocal lattice plot
    k_out = rot2d(two_theta)@k_in
    k = k_in - k_out
    scatter_origin = np.array([0,0])
    axes[1].arrow(scatter_origin[0]+k_in[0], scatter_origin[1]+k_in[1], -k_in[0], -k_in[1],
                  lw=lw, head_width=head_width, head_length=head_length, width=0.05,
                  length_includes_head=True,
                  fc='red', ec='red', label='$k_{in}}$')
    axes[1].arrow(scatter_origin[0]+k_in[0], scatter_origin[1]+k_in[1], -k_out[0], -k_out[1],
                  lw=lw, head_width=head_width, head_length=head_length, width=0.05,
                  length_includes_head=True,
                  fc='orange', ec='orange', label='$k_{out}$')
    axes[1].arrow(scatter_origin[0], scatter_origin[1], k[0], k[1],
                  lw=lw, head_width=head_width, head_length=head_length, width=0.05,
                  length_includes_head=True,
                  fc='black', ec='black', label='$k$')

    circle_theta = np.linspace(-np.pi, np.pi, 200)
    r = 2*np.pi/wavelength/(2*np.pi)
    axes[1].plot(np.sin(circle_theta)*r + scatter_origin[0]+k_in[0],
            np.cos(circle_theta)*r + scatter_origin[1]+k_in[1],
            color='red', label="Ewald circle")
    r = np.linalg.norm(k)
    axes[1].plot(np.sin(circle_theta)*r + scatter_origin[0],
            np.cos(circle_theta)*r + scatter_origin[1],
            color='black', label="$|\mathbf{k}|$ circle")    
    axes[1].legend()

    #ax.set_aspect('equal')

In [ ]:
fig_ax = plt.subplots(1, 2, figsize=(7.5,3.8), tight_layout=True)
ex10_wp = WidgetPlot(plot_braggs_reflection, 
               WidgetParbox(a11 = (1., -4, 4, 0.1, r'$a_{11} / Å$'),
                            a12 = (0., -4, 4, 0.1, r'$a_{12} / Å$'),
                            a21 = (0., -4, 4, 0.1, r'$a_{21} / Å$'),
                            a22 = (2., -4, 4, 0.1, r'$a_{22} / Å$'),
                            phi = (0., 0., 2*np.pi, 0.05, r'$\phi$'),
                            wavelength = (1., 0.2, 2, 0.05, r'$\lambda$ / Å'),
                            two_theta = (1., 0.1, np.pi, 0.05, r'$2\theta$')
                           ),
                     fig_ax=fig_ax)
display(ex10_wp)

You will notice that it is not easy, with a fixed position of the incoming light, to orient the detector so that $\mathbf{k}$ corresponds to a reciprocal lattice vector. The widget above allows you to change the orientation of the direct lattice. Note how, by changing the orientation of the crystal, you can bring any of the reciprocal lattice point that lie on the circle with radius $k$ in coincidence with the scattering wavevector $\mathbf{k}$.

<span style="color:blue"> **11** What happens if you consider scattering from a collection of crystals with different orientation (a powder sample)? Write the relation between the _magnitude_ of reciprocal lattice vectors $G$ and the corresponding scattering angle $\theta$.
</span>

In [ ]:
ex11_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex11-answer", ex11_txt, "value")
display(ex11_txt)

# Diffraction from an atomic structure

An actual crystalline structure involves both a lattice and of an a-periodic basis $\{\mathbf{s}_i\}$ of atoms. Each will scatter with a form factor $f_i$ (that depends on the modulus of $\mathbf{k}$ but we will take to be a constant proportional to the atomic charge $Z_i$). The scattered amplitude can be written as  

$$
F(\mathbf{k}) = \frac{1}{N} \sum^N_{j \textrm{ atom}} f_j\exp(-\mathrm{i}\mathbf{k}\cdot\mathbf{r}_j)\textrm{, with }\mathbf{r}_j\textrm{ position of atom }j
$$

by breaking the position of atoms into lattice vector and basis positions, we can write 

$$
F(\mathbf{k}) = \frac{1}{n_\mathrm{cell} n_\mathrm{basis}} 
\sum^{n_\mathrm{cell}}_{\mathbf{T}} \exp(-\mathrm{i}\mathbf{k}\cdot \mathbf{T}) 
\sum_m^{n_\mathrm{basis}} f_m \exp(-\mathrm{i}\mathbf{k}\cdot\mathbf{s}_m).
$$

One sees that the sum over lattice vectors selects the wavevectors that correspond to a reciprocal lattice vector $\mathbf{G}$, while the sum over the atomic basis modulates the amplitude of the peak through a _structure factor_

$$
F_\mathbf{G} = \sum_m^{n_\mathrm{basis}} f_m \exp(-\mathrm{i}\mathbf{G}\cdot\mathbf{s}_m).
$$


In [ ]:
#TODO AG: I think here the 2D scattering is too messy. Let's plot the intensities 
# as bars as a function of theta
#QUESTION@michele not sure what you mean exactly here. I just removed the noise and supercell part
#                 I think it would be better, if we use for all diffraction exercises the
#                 same structure. So if they want, they can look at the same structure with
#                 how the reflection behave and how the diffraction pattern.

We will compute the structure factor for InAs. You can have a look at the real and reciprocal lattice.

In [ ]:
in_as = ase.Atoms("InAs", positions=[[0,0,0],[0.5,0.5,0]], cell=[1,1,1], pbc=[True,True,False])
in_as.positions *= 3
in_as.cell *= 3

In [ ]:
def plot_reciprocal_and_real_lattice_InAs(axes, phi):
    def rot2d(angle):
        return np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
    a1 = in_as.cell[0][:2]
    a2 = in_as.cell[1][:2]
    
    b1, b2 = ex09_wci.get_function_object()(a1, a2)

    A = np.array([a1, a2])
    B = np.array([b1, b2])
    
    # ! has to be mod 2 integer !
    lattice_size = 120
    # frac coords of atom1
    frac_coords = np.linalg.inv(A) @ in_as.positions[1][:2]
    real_lattice_atom0 = (np.mgrid[:lattice_size,:lattice_size].T @ A).reshape(-1, 2)
    real_lattice_atom1 = (np.mgrid[:lattice_size,:lattice_size].T @ A + frac_coords @ A).reshape(-1, 2)    
    real_lattice_atom0 -= (np.array([lattice_size//2,lattice_size//2]) @ A).reshape(-1, 2)
    real_lattice_atom1 -= (np.array([lattice_size//2,lattice_size//2]) @ A).reshape(-1, 2)
    
    rot_phi = rot2d(phi)
    real_lattice_atom0 = real_lattice_atom0 @ rot_phi
    real_lattice_atom1 = real_lattice_atom1 @ rot_phi
    A = A @ rot_phi.T

    reciprocal_lattice_atom0 = (np.mgrid[:lattice_size,:lattice_size].T @ B).reshape(-1, 2)
    reciprocal_lattice_atom1 = (np.mgrid[:lattice_size,:lattice_size].T @ B + frac_coords @ B).reshape(-1, 2)    
    reciprocal_lattice_atom0 -= (np.array([lattice_size//2,lattice_size//2]) @ B).reshape(-1, 2)
    reciprocal_lattice_atom1 -= (np.array([lattice_size//2,lattice_size//2]) @ B).reshape(-1, 2)
    
    reciprocal_lattice_atom0 = reciprocal_lattice_atom0 @ rot_phi
    reciprocal_lattice_atom1 = reciprocal_lattice_atom1 @ rot_phi
    B = B @ rot_phi.T

    
    s = 35
    axes[0].scatter(real_lattice_atom0[:,0], real_lattice_atom0[:,1], color='red', s=s)
    axes[0].scatter(real_lattice_atom1[:,0], real_lattice_atom1[:,1], color='pink', s=s)
    axes[1].scatter(reciprocal_lattice_atom0[:,0], reciprocal_lattice_atom0[:,1], color='darkblue', s=s)
    axes[1].scatter(reciprocal_lattice_atom1[:,0], reciprocal_lattice_atom1[:,1], color='lightblue', s=s)

    head_length = 0.5
    head_width = 0.7
    width = 0.25
    lw = 3
    alpha_unit_cell_vectors = 1.0

    axes[1].arrow(0, 0, B[0,0], B[1,0], lw=lw,
             head_width=head_width, head_length=head_length,
             width=width,
             alpha=alpha_unit_cell_vectors,
             length_includes_head=True,
             fc='darkblue', ec='black')

    axes[1].arrow(0, 0, B[0,1], B[1,1], lw=lw,
             head_width=head_width, head_length=head_length,
             width=width,
             alpha=alpha_unit_cell_vectors,
             length_includes_head=True,
             fc='darkblue', ec='black')

    
    axes[0].arrow(0, 0, A[0,0], A[1,0], lw=lw,
             head_width=head_width, head_length=head_length,
             width=width-0.1,
             alpha=alpha_unit_cell_vectors,
             length_includes_head=True,
             fc='pink', ec='palevioletred')

    axes[0].arrow(0, 0, A[0,1], A[1,1], lw=lw,
             head_width=head_width, head_length=head_length,
             width=width-0.05,
             alpha=alpha_unit_cell_vectors,
             length_includes_head=True,
             fc='pink', ec='palevioletred')

    axes[0].set_title('real space')
    axes[0].set_xlim(-10,10)
    axes[0].set_ylim(-10,10)
    axes[0].set_xlabel("$x$ / $\AA$")
    axes[0].set_ylabel("$y$ / $\AA$")
    
    axes[1].set_title('reciprocal space')
    axes[1].set_xlim(-10,10)
    axes[1].set_ylim(-10,10)
    axes[1].set_xlabel("$x$ / $\AA$")
    axes[1].set_ylabel("$y$ / $\AA$")

In [ ]:
fig_ax = plt.subplots(1, 2, figsize=(9,4.2), tight_layout=True)
inas_wp = WidgetPlot(plot_reciprocal_and_real_lattice_InAs, 
               WidgetParbox(
                            phi = (0., 0., 2*np.pi, 0.05, r'$\phi$'),
                           ),
                     fig_ax=fig_ax)
display(inas_wp)

In [ ]:
# set upt the code widget window
ex12_wci = WidgetCodeInput(
        function_name="diffraction_pattern", 
        function_parameters="k, positions, atomic_numbers",
        docstring="""
Computes the scatter intensity for the scattering vector k.
Each atom is a scattering center.

:param k: scattering vector k = k_in-k_out
:param positions: positions of atoms in lattice, array of dimension N x 3
:param atomic_numbers: atomic numbers Z of atoms in lattice, array of length N
""",
    function_body="""
from numpy import abs as absolute
from numpy import exp

# you can define an imaginary number in python with 1j
# for example
# a = 1j # i
# a = 1+1j # 1+i
# b = absolute(a)**2 # |a|^2 = aa*

structure_factor = 0

# you can access the atomic number Z as
for j in range(len(atomic_numbers)):
    Z = atomic_numbers[j] # atom number of atom j
    r_j = positions[j] # atom position of atom j
    #structure_factor += ...

# write solution here

### BEGIN SOLUTION
for j in range(len(atomic_numbers)):
    # approx of atom form factors 
    f_j = atomic_numbers[j]
    r_j = positions[j]
    structure_factor += exp(-1j*k @ r_j)*f_j
scattered_intensity = absolute(structure_factor)
scattered_intensity /= len(atomic_numbers)
### END SOLUTION

return scattered_intensity
"""
        )

data_dump.register_field("ex12-function", ex11_wci, "function_body")

In [ ]:
def add_periodic_image(structure, unit_cell_structure, direction):
    cell_length = unit_cell_structure.cell[0][0]
    periodic_image = copy.deepcopy(unit_cell_structure)
    periodic_image.positions += np.array(direction)*cell_length
    structure += periodic_image

def setup_structure(structure, M, noise_sigma):
    import itertools
    supercell = ase.Atoms()
    structure = copy.deepcopy(structure)
    for direction in itertools.product(list(range(M+1)), repeat=2):
        add_periodic_image(supercell, structure, list(direction) + [0])

    np.random.seed(0)
    supercell.positions[:, :2] += np.random.normal(size=(len(supercell), 2), scale=noise_sigma)
    return supercell

def rot2d(angle):
    return np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])

def plot_diffraction_pattern(ax, wavelength):
    M = 7
    phi = 0
    noise_sigma = 0
    structure = ase.Atoms("InAs", positions=[[0,0,0],[0.5,0.5,0]], cell=[1,1,0], pbc=[True,True,False])
    structure.positions *= 3
    structure.cell *= 3

    supercell = setup_structure(structure, M, noise_sigma)
    supercell.positions[:,:2] = supercell.positions[:, :2] @ rot2d(phi)
    
    theta = np.linspace(0.,np.pi,200)
    n = np.array([np.sin(theta),np.cos(theta),0])
    k = 2*np.pi*n/wavelength
    
    scattered_intensity = ex12_wci.get_function_object()(k, supercell.positions, supercell.numbers)
    
    ax.plot(theta*180/np.pi, scattered_intensity)
    
    ax.set_xlim(0,180)
    ax.set_xlabel("theta $\\theta$ [degree]")
    ax.set_ylim(-0.5,42) 
    ax.set_ylabel("Scattering amplitude $F(\mathbf{q}(\\theta))$")

In [ ]:
fig_ax = plt.subplots(1, 1, figsize=(7,6), tight_layout=True)
ex12_wp = WidgetPlot(plot_diffraction_pattern, 
               WidgetParbox(
                            wavelength = (1., 0.2, 2*np.pi, 0.05, r'$\lambda$')),
                             fig_ax=fig_ax)
ex12_wcc = WidgetCodeCheck(ex12_wci, ref_values = {}, demo = ex12_wp)
display(ex12_wcc)